In [1]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))
    
from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data, rel_disp, third_moment
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3,  x_log_of_pn
from MPyDATA_examples.utils.show_plot import show_plot
import numpy as np
import matplotlib
from matplotlib import pyplot

In [3]:
from MPyDATA_examples.Olesik_et_al_2020.setup import default_opt_set, colors, default_mixing_ratios_g_kg

In [10]:
plot_steps = [0, 2, 5]
def compute_and_plot(psi_coord, grid_layout, n_bins, GC_max, variants, plots, filename):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        GC_max = GC_max,
        opt_set=tuple(default_opt_set[variant] for variant in variants)
    )
    return_value = {}
    for coord in results.keys():
        out_steps = results[coord]['grid']['out_steps']
        dt = results[coord]['grid']['dt']
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()):
            variant = variants[opt_i]
            plot_data = results[coord]['numerical'][opts]
            for i in range(len(out_steps)):
                mnorm = setup.mixing_ratios[i]
                t = out_steps[i] * dt
                linewidth = 1+1.5*i/len(out_steps)
                if opt_i == 0 and i in plot_steps:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='discretised analytical solution (actual dotted)', 
                        mnorm=mnorm,
                        color='black',
                        linewidth = linewidth
                    )
                    
                str_repl = [["'n_iters': 1","upwind"],
                            ["'n_iters': 2","MPDATA 2 iterations"],
                            ["'n_iters': 3","MPDATA 3 iterations"],
                            ["'",""],
                            [": True",""],
                            ["_", " "],
                            ["{",""],["}",""],[",",""],["flux corrected transport","non-oscillatory"]]                            
                for repl in str_repl:
                    opts = opts.replace(repl[0], repl[1])
                    
                if variant not in return_value:
                    return_value[variant] = {'R_d': [], 'R_m': [], 'label': opts}

                if i in [0, 2, 5]: 
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        plot_data[i], 
                        label=opts, 
                        bin_boundaries=results[coord]['grid']['rh'], 
                        linewidth = linewidth,
                        mnorm=mnorm, color = colors[variant], fill=False
                    )
                
                dp_dr = psi_coord.dx_dr(results[coord]['grid']['r'])
                numeric_rel_d = rel_disp(results[coord]['grid']['rh'], plot_data[i]/dp_dr, psi_coord)
                analytic_rel_d = rel_disp(results[coord]['grid']['rh'], results[coord]['analytical'][i]/dp_dr, psi_coord)
                dispersion_ratio = (numeric_rel_d / analytic_rel_d - 1) * 100
#                 print(f"dispersion ratio excess {opts}: {dispersion_ratio.to(setup.si.dimensionless).magnitude:.2g} %")
                return_value[variant]['R_d'].append(dispersion_ratio)
                                
                numeric_mass = third_moment(results[coord]['grid']['rh'], plot_data[i]/dp_dr, psi_coord, normalised=False)
                analytic_mass = third_moment(results[coord]['grid']['rh'], results[coord]['analytical'][i]/dp_dr, psi_coord, normalised=False)
                mass_ratio = (numeric_mass / analytic_mass - 1) * 100
#                 print(f"mass ratio excess: {mass_ratio.to(setup.si.dimensionless).magnitude:.2g} %")
                return_value[variant]['R_m'].append(mass_ratio)

        pyplot.legend(bbox_to_anchor=(0.5, -0.25), loc='upper center')
        show_plot(filename)
    return return_value

In [11]:
n_bins = 75
plot_setup_radius_square_mass_doubling = {"psi_coord":x_p2(), "grid_layout":x_log_of_pn(r0=1, base=2), "GC_max":.26,"n_bins":n_bins}
plot_setup_radius_linear = {"psi_coord":x_id(), "grid_layout":x_id(), "GC_max":.07, "n_bins":n_bins}
plot_setup_default = plot_setup_radius_square_mass_doubling
table_data = []

In [12]:
compute_and_plot(**plot_setup_radius_linear, variants=('a',), plots=['n','m'],  filename = 'fig_upwindb.pdf');

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    4.6s finished
No handles with labels found to put in legend.


In [13]:
table_data.append(compute_and_plot(**plot_setup_default, variants=('a',), plots=['n','m'], filename = 'fig_upwinda.pdf'))

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    4.4s finished
No handles with labels found to put in legend.


In [14]:
table_data.append(compute_and_plot(**plot_setup_default, variants=('a','b','g'), plots=['n'], filename = 'fig_mpdatas.pdf'))

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   32.6s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   32.6s finished


In [15]:
table_data.append(compute_and_plot(**plot_setup_default, variants=('b', 'c'), plots=['n'], filename = 'fig_iga.pdf'))

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   22.1s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   22.1s finished


In [28]:
table_data.append(compute_and_plot(**plot_setup_default, variants=('c','d'), plots=['n'], filename = 'fig_iga_fct.pdf'))

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   26.1s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   26.1s finished


In [29]:
table_data.append(compute_and_plot(**plot_setup_default, variants=('d','e'), plots=['n'], filename = 'fig_iga_fct_dpdc.pdf'))

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   38.9s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   38.9s finished


In [30]:
table_data.append(compute_and_plot(**plot_setup_default, variants=('g','f'), plots=['n'], filename = 'fig_tot.pdf'))

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   42.9s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:   42.9s finished


In [31]:
table_data.append(compute_and_plot(**plot_setup_default, variants=('f','h'), plots=['n'], filename = 'fig_multiopt.pdf'))

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   2 out of   2 | elapsed:  1.1min finished


In [16]:
print(table_data)

[{'a': {'R_d': [<Quantity(0.0, 'dimensionless')>, <Quantity(7.255371227953278, 'dimensionless')>, <Quantity(24.371303922636823, 'dimensionless')>, <Quantity(40.87059574869934, 'dimensionless')>, <Quantity(56.818596357206005, 'dimensionless')>, <Quantity(70.76346306222592, 'dimensionless')>], 'R_m': [<Quantity(0.0, 'dimensionless')>, <Quantity(3.517270695496788, 'dimensionless')>, <Quantity(5.528419489859826, 'dimensionless')>, <Quantity(5.680578382176993, 'dimensionless')>, <Quantity(4.903454296304632, 'dimensionless')>, <Quantity(4.409061522747049, 'dimensionless')>], 'label': 'upwind'}}, {'a': {'R_d': [<Quantity(0.0, 'dimensionless')>, <Quantity(7.255371227953278, 'dimensionless')>, <Quantity(24.371303922636823, 'dimensionless')>, <Quantity(40.87059574869934, 'dimensionless')>, <Quantity(56.818596357206005, 'dimensionless')>, <Quantity(70.76346306222592, 'dimensionless')>], 'R_m': [<Quantity(0.0, 'dimensionless')>, <Quantity(3.517270695496788, 'dimensionless')>, <Quantity(5.528419489

In [57]:
plot_data = {}
for datum in table_data:
    for key, value in datum.items():
        plot_data[key] = value
        
def plot_measures(measure, ax, legend=False): 
    for variant, values in plot_data.items():
        ax.scatter(default_mixing_ratios_g_kg, values[measure], label=values['label'], color=colors[variant])
        ax.plot(default_mixing_ratios_g_kg, values[measure], color='gray', linestyle='dotted')
    ax.grid()
    ax.set_ylabel(f'${measure}$ [%]')
    ax.set_xlabel('mixing ratio [g kg$^{-1}$]')
    if legend:
        pyplot.legend(bbox_to_anchor=(-0.1, -0.15), loc='upper center')
fig, axs = pyplot.subplots(1,2, figsize=(15,9))

plot_measures('R_d', axs[0])
plot_measures('R_m', axs[1], legend=True)

show_plot('measures.pdf')

C:\Users\Michael\anaconda3\lib\site-packages\numpy\core\_asarray.py:138: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
C:\Users\Michael\anaconda3\lib\site-packages\numpy\core\_asarray.py:138: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
C:\Users\Michael\anaconda3\lib\site-packages\numpy\core\_asarray.py:138: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
C:\Users\Michael\anaconda3\lib\site-packages\numpy\core\_asarray.py:138: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
C:\Users\Michael\anaconda3\lib\site-packages\numpy\core\_asarray.py:138: UnitStrippedWarning: The unit of the qu